# Steganography

## Module Import

In [9]:
import wave
import numpy as np
import os
from pydub import AudioSegment

## Audio Conversion Function

Now we define a function that check the imported cover audio if it has '.wav' extension. If not, we convert it using PyDub.

In [12]:
def convertToWav(inputFile):
    if not os.path.exists(inputFile):
        print(f"Error: The input file '{inputFile}' does not exist.")
        return None
    _, ext = os.path.splitext(inputFile)
    if ext.lower() == '.wav':
        print(f"The file '{inputFile}' is already in WAV format.")
        return inputFile
    outputFile = os.path.splitext(inputFile)[0] + '.wav'
    try:
        audio = AudioSegment.from_file(inputFile)
        audio.export(outputFile, format="wav")
        print(f"Audio successfully converted to WAV format: {outputFile}")
        return outputFile
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return None

We do the same function on the message audio.

## Capacity Checking Function

As cover audio can hold only so much of message audio, it is crucial to check for overflow prior performing steganography.

In [14]:
def calculate_audio_capacity(file_path):
    """Calculate the bit capacity of the cover audio."""
    with wave.open(file_path, 'rb') as wf:
        # Extract properties
        n_channels = wf.getnchannels()          # Number of channels
        sample_width = wf.getsampwidth()       # Bytes per sample
        frame_rate = wf.getframerate()         # Sampling rate (samples per second)
        n_frames = wf.getnframes()             # Total frames
        bit_depth = sample_width * 8           # Bits per sample
        duration = n_frames / frame_rate       # Duration in seconds

        # Calculate total number of samples
        total_samples = n_frames * n_channels  # Each channel counts as a sample

        # Capacity is 1 bit per sample for LSB steganography
        capacity_bits = total_samples          # Total bits available for LSB

        print(f"Cover Audio Properties:")
        print(f"  Duration: {duration:.2f} seconds")
        print(f"  Channels: {n_channels}")
        print(f"  Bit Depth: {bit_depth} bits")
        print(f"  Frame Rate: {frame_rate} Hz")
        print(f"  Total Samples: {total_samples}")
        print(f"  Capacity for LSB Steganography: {capacity_bits} bits")

        return capacity_bits, duration


def calculate_message_size(file_path):
    """Calculate the bit size of the message audio."""
    with wave.open(file_path, 'rb') as wf:
        # Extract properties
        n_channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        frame_rate = wf.getframerate()
        n_frames = wf.getnframes()
        bit_depth = sample_width * 8           # Bits per sample
        duration = n_frames / frame_rate       # Duration in seconds

        # Total number of samples in the message audio
        total_samples = n_frames * n_channels

        # Total bits in the message
        total_bits = total_samples * bit_depth

        print(f"Message Audio Properties:")
        print(f"  Duration: {duration:.2f} seconds")
        print(f"  Channels: {n_channels}")
        print(f"  Bit Depth: {bit_depth} bits")
        print(f"  Frame Rate: {frame_rate} Hz")
        print(f"  Total Samples: {total_samples}")
        print(f"  Message Size: {total_bits} bits")

        return total_bits


def check_capacity(cover_audio_path, message_audio_path):
    """Check if the message audio can fit into the cover audio."""
    if not os.path.exists(cover_audio_path):
        print(f"Error: Cover audio file '{cover_audio_path}' not found.")
        return False
    if not os.path.exists(message_audio_path):
        print(f"Error: Message audio file '{message_audio_path}' not found.")
        return False

    # Calculate cover audio capacity
    cover_capacity, cover_duration = calculate_audio_capacity(cover_audio_path)

    # Calculate message audio size
    message_size = calculate_message_size(message_audio_path)

    # Compare capacities
    if cover_capacity >= message_size:
        print(f"\nSuccess: The message audio can fit into the cover audio!")
        print(f"  Cover Audio Capacity: {cover_capacity} bits")
        print(f"  Message Audio Size: {message_size} bits")
        return True
    else:
        print(f"\nError: The message audio is too large to fit into the cover audio.")
        print(f"  Cover Audio Capacity: {cover_capacity} bits")
        print(f"  Message Audio Size: {message_size} bits")
        print(f"  You would need a larger cover audio or a smaller message audio.")
        return False